In [130]:
import numpy as np
import torch
from torch import nn

# BAI 1

In [131]:
def add_zeros(input, s):
    z = s - 1
    input_sz = input.shape[0]
    pos = np.arange(1, input_sz)
    pos = np.repeat(pos, z)

    zero_inserted_input = np.insert(input, pos, 0, axis=1)
    zero_inserted_input = np.insert(zero_inserted_input, pos, 0, axis=0)

    new_input = zero_inserted_input
    return new_input

def convolve(input, kernel, output_sz):
    convolved = np.zeros((output_sz, output_sz))
    kernel_sz = kernel.shape[0]
    for i in range(output_sz):
        for j in range(output_sz):
            sub_matrix = input[i:i+kernel_sz, j:j+kernel_sz]
            convolved[i, j] = np.sum(np.multiply(sub_matrix, kernel))
    return convolved
    
def transposed_conv(input, kernel, s, p):
    input_sz = input.shape[0]
    kernel_sz = kernel.shape[0]
    output_sz = (input_sz - 1) * s + kernel_sz - 2 * p
    new_input = add_zeros(input, s)
    return convolve(new_input, kernel, output_sz)


In [132]:
test_input = np.array([[2, 4, 4],
                       [0, 1, 3],
                       [1, 2, 4]])

test_kernel = np.array([[3, 1],
                        [1, 5]])


transpose_convolution = transposed_conv(
    test_input, test_kernel, 2, 1)

print('transpose conv:\n', transpose_convolution)
print('\n')
print('added_zeros:\n',add_zeros(test_input, 2))
print('\n')
print('kernel:\n', test_kernel)


transpose conv:
 [[ 6.  4. 12.  4.]
 [ 0.  5.  1. 15.]
 [ 0.  1.  3.  3.]
 [ 1. 10.  2. 20.]]


added_zeros:
 [[2 0 4 0 4]
 [0 0 0 0 0]
 [0 0 1 0 3]
 [0 0 0 0 0]
 [1 0 2 0 4]]


kernel:
 [[3 1]
 [1 5]]


In [2]:
0.5e-6

5e-07